In [ ]:
# !pip install matplotlib
# !pip install seaborn
# !pip install pandas
# !pip install scikit-image
# !pip install -U scikit-learn




# Bibliotecas

In [ ]:
import os

import numpy as np
import math

from skimage import util, transform, filters, color, measure, morphology
from sklearn import model_selection, neighbors, metrics, preprocessing

from skimage import util, transform, filters, color, measure, morphology
from sklearn import model_selection, neighbors, metrics, preprocessing
from skimage.measure import label as skimage_label, regionprops
from skimage.morphology import remove_small_objects, remove_small_holes
from sklearn.neighbors import RadiusNeighborsClassifier

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

### %matplotlib notebook
### %matplotlib widget

# Dataset

In [ ]:
# Local do Dataset (6 Classes)
dataset = "dataset"

In [ ]:
# Lista das pastas na pasta 'dataset' (nomes das classes)
classes_list = os.listdir(dataset)

# Lista com as imagens no dataset
image_list = []

# Lista com os rótulos das imagens
label_list = []

# Lista com os nomes das imagens
filename_list_ = []

# Percorre as classes do dataset
for classe in classes_list:
    
    # Listagem de todas as imagens na pasta daquela classe
    filename_list = os.listdir(os.path.join(dataset, classe))
    
    # Percorre os arquivos na pasta atual
    for filename in filename_list:
        # Carrega a imagem
        img_temp = plt.imread(os.path.join(dataset, classe, filename))
        
        # Redimensiona a imagem para 1/4 do tamanho original
        img_temp = transform.resize(img_temp, (img_temp.shape[0]//4, img_temp.shape[1]//4), anti_aliasing=True)
        
        # Adiciona a imagem a lista de imagens
        image_list.append(img_temp)
        
        # Adiciona o rótulo da imagem à lista de rótulos
        label_list.append(classe)
        
        # Adiciona o nome da imagem à uma lista (para fins de visualização)
        filename_list_.append(filename)

In [ ]:
# Coverte para numpy array
label_list = np.array(label_list)
# Lista com os rótulos das imagens        
print(label_list)

## Plotando imagens do Dataset

In [ ]:
# Seleciona apenas as primeiras N imagens de cada classe
image_list_temp = []
filename_list_temp = []

# Itera pelo número de classes
for i, class_name in enumerate(classes_list):
    print(i, class_name)
    # As 6 primeiras ocorrencias em que em label_list igual a class_name.
    # Imagem.
    image_list_temp += [image_list[j] for j in np.where(label_list==class_name)[0][:6]]
    # Nome do arquivo.
    filename_list_temp += [filename_list_[j] for j in np.where(label_list==class_name)[0][:6]]
    

In [ ]:
# Número de imagens
num_images = len(image_list_temp)

# Calcular as dimensões da grade
rows = math.ceil(num_images / 6)
cols = 6

# Criar a figura com grade ajustada
fig, ax = plt.subplots(rows, cols, figsize=(9, 5))

# Garantir que 'ax' seja bidimensional
if rows == 1:
    ax = ax[np.newaxis, :]  # Ajuste para uma única linha de subplots

# Iterar pelas imagens e títulos
for i, (image, filename) in enumerate(zip(image_list_temp, filename_list_temp)):
    # Normalizar se necessário
    if image.max() > 1:  # Assumindo que já está na escala [0, 255]
        normalized_image = image
    else:  # Escala [0, 1]
        normalized_image = (image * 255).astype('uint8')

    # Plotar imagem
    ax[i // 6, i % 6].imshow(normalized_image, cmap='gray', vmin=0, vmax=255)
    ax[i // 6, i % 6].set_title(str(filename))
    ax[i // 6, i % 6].axis('off')  # Opcional: desativar os eixos

# Remover subplots vazios
for i in range(num_images, rows * cols):
    fig.delaxes(ax[i // 6, i % 6])

fig.tight_layout()
plt.show()

# Extração de Features

In [ ]:
# Nomes das caracteristicas computadas
features = ['area', 'major_axis', 'minor_axis', 'solidity', 'eccentricity']

In [ ]:
# Inicializa as listas para armazenar as características e os rótulos filtrados
feature_mat = []
filtered_labels = []

# Itera pelos pares de imagens segmentadas e seus rótulos
for i, (image, label) in enumerate(zip(image_list, label_list)):
    # Validar o tipo da entrada
    if not isinstance(image, np.ndarray):
        print(f"Erro: a entrada não é uma matriz NumPy na imagem {i}. Tipo recebido: {type(image)}")
        continue

    # Limpeza inicial da máscara
    seg_image_cleaned = remove_small_objects(image > 0, min_size=50)
    seg_image_cleaned = remove_small_holes(seg_image_cleaned, area_threshold=50)

    # Garantir formato binário
    seg_image_cleaned = np.asarray(seg_image_cleaned, dtype=bool)

    # Rotula as regiões conectadas usando a função correta
    im_lbl = skimage_label(seg_image_cleaned)

    # Verifica se há regiões conectadas
    props = regionprops(im_lbl)
    if len(props) == 0:
        print(f'Imagem {i} - ERRO: Nenhum objeto detectado após limpeza.')
        continue

    # Filtra apenas o maior objeto
    largest_region = max(props, key=lambda x: x.area)

    # Extrai características do maior objeto
    area = largest_region.area
    major_axis = largest_region.major_axis_length
    minor_axis = largest_region.minor_axis_length
    solidity = largest_region.solidity
    eccentricity = largest_region.eccentricity

    # Adiciona as características extraídas à lista de características
    feature_list = [area, major_axis, minor_axis, solidity, eccentricity]
    feature_mat.append(feature_list)

    # Adiciona o rótulo correspondente à lista de rótulos filtrados
    filtered_labels.append(label)

# Cria o DataFrame com as características e os rótulos
features = ['area', 'major_axis', 'minor_axis', 'solidity', 'eccentricity']
df = pd.DataFrame(feature_mat, columns=features)
df['class'] = filtered_labels

# Exibe o DataFrame resultante
display(df)

In [ ]:
# Converte a lista de caracteristicas para um arranjo NumPy
feature_map = np.array(feature_mat)

# Imprime a matriz de caracteristica
with np.printoptions(precision=4, suppress=True):
    print(feature_map.shape)

In [ ]:
# Algumas estatisticas sobre o conjunto de caracteristicas
with np.printoptions(precision=4, suppress=True):
    print(feature_map.min(0))
    print(feature_map.max(0))
    print(feature_map.mean(0))
    print(feature_map.std(0))

# Plotando as Caracteristicas coletadas

In [ ]:
g = sns.PairGrid(df, hue='class', vars=features)
g.fig.set_size_inches(8, 8)
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)
g.add_legend()

# Validação Cruzada - Hold-out

In [ ]:
import numpy as np

# Converte feature_mat para um array NumPy
feature_mat_np = np.array(feature_mat)

# Seleciona apenas as duas primeiras características (Área e maior eixo)
feature_mat_ok = feature_mat_np[:, [0, 3]]

# Exibe as características selecionadas
print(feature_mat_ok)

In [ ]:
# Separa o conjunto de dados em 'feature_map_ok', de acordo com 'label_list'. 
# 30% das imagens vão para o conjunto de testes.
X_train, X_test, y_train, y_test = model_selection.train_test_split(feature_mat_ok, 
                                                                    label_list, 
                                                                    test_size=0.3, 
                                                                    random_state=42,
                                                                    stratify = label_list                                                                   
                                                                   )

## Normalizando as caracteristicas

In [ ]:
# Média das caracteristicas do conjunto de treinamento
X_train_mean = X_train.mean(0)
# Desvio padrão das caracteristicas do conjunto de treinamento
X_train_std = X_train.std(0)

with np.printoptions(precision=4, suppress=True):
    print(X_train.mean(0))
    print(X_train.std(0))

In [ ]:
# Transformada Normal de Caracteristicas
X_train_norm = (X_train - X_train_mean) / X_train_std
X_test_norm = (X_test - X_train_mean) / X_train_std

with np.printoptions(precision=4, suppress=True):
    print(X_train_norm.mean(0))
    print(X_test_norm.mean(0))

# Classificando usando K-NN

In [ ]:
# Constrói um classificador do tipo K-NN
# K = 5
clf = neighbors.KNeighborsClassifier(n_neighbors=3)

# Treinando o classificador
clf.fit(X_train_norm, y_train)

# Testando o classificador
pred = clf.predict(X_test_norm)

# Avaliação do modelo

In [ ]:
# Acertos
acertos = y_test == pred

print('\n Predição:')
print(pred)
print('\nReal:')
print(y_test)
print('\nAcerto/Erro:')
print(acertos.astype(int))

## Matriz de confusão

In [ ]:
print('\nMatriz de confusão:')
print(metrics.confusion_matrix(y_test, pred))

print('\nRelatório de classificação:')
print(metrics.classification_report(y_test, pred))

# Classificador R-NN

In [ ]:
# Constrói um classificador do tipo R-NN
# Raio = 1.0 (valor pode ser ajustado com base nos dados)
clf = RadiusNeighborsClassifier(radius=1)

# Treinando o classificador
clf.fit(X_train_norm, y_train)

# Testando o classificador
pred = clf.predict(X_test_norm)

In [ ]:
# Acertos
acertos = y_test == pred

print('\n Predição:')
print(pred)
print('\nReal:')
print(y_test)
print('\nAcerto/Erro:')
print(acertos.astype(int))

In [ ]:
print('\nMatriz de confusão:')
print(metrics.confusion_matrix(y_test, pred))

print('\nRelatório de classificação:')
print(metrics.classification_report(y_test, pred))